In [ ]:
# | default_exp sms_utils

In [ ]:
from airt.testing import activate_by_import

[INFO] airt.testing.activate_by_import: Testing environment activated.
[INFO] numexpr.utils: Note: NumExpr detected 64 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[INFO] numexpr.utils: NumExpr defaulting to 8 threads.
[INFO] airt.keras.helpers: Using a single GPU #0 with memory_limit 1024 MB


In [ ]:
# | export

import json
import os
from time import sleep
from typing import *

import numpy as np
import requests
from airt.logger import get_logger
from fastapi import HTTPException, status
from sqlmodel import Session, select

import airt_service
import airt_service.sanitizer
from airt_service.db.models import SMS, SMSProtocol, User
from airt_service.errors import ERRORS
from airt_service.helpers import commit_or_rollback

In [ ]:
from contextlib import contextmanager

import pyotp
import pytest
from _pytest.monkeypatch import MonkeyPatch

from airt_service.db.models import (
    User,
    create_user_for_testing,
    get_session_with_context,
)
from airt_service.users import (
    ActivateMFARequest,
    activate_mfa,
    generate_mfa_url,
    send_sms_otp,
)

[INFO] airt.executor.subcommand: Module loaded.


In [ ]:
# | exporti

logger = get_logger(__name__)

In [ ]:
logger.info("log a random string")

[INFO] __main__: log a random string


In [ ]:
# | exporti


def _generate_application_name() -> str:
    """Generate an application template name

    The generated template name will be used to create an application template which is necessary for
    using the infobip 2FA service. These templates will be created only once and will be reused on
    subsequent requests.

    Returns:
        The application template name
    """
    domain = os.environ["DOMAIN"]

    return os.environ["HOSTNAME"] if domain == "localhost" else domain

In [ ]:
@contextmanager
def change_domain(domain_name: str):
    try:
        domain = os.environ["DOMAIN"]
        os.environ["DOMAIN"] = domain_name
        yield
    finally:
        os.environ["DOMAIN"] = domain

In [ ]:
expected = "random_domain"
with change_domain(expected):
    actual = _generate_application_name()
    display(actual)
    assert actual == expected


actual = _generate_application_name()
expected = os.environ["HOSTNAME"]
display(actual)
assert actual == expected

'random_domain'

'harish-airt-service-devel'

In [ ]:
# | export


def get_application_and_message_config() -> Dict[str, Dict[str, Any]]:
    """Get the application and message template configurations

    Before using the Infobip's 2FA service it is necessary to configure the properties and templates for the use case. Properties,
    such as the number of allowed PIN attempts and PIN time to live etc., are configured by creating an application. Templates,
    such as message text, PIN type, PIN length etc., are configured by creating a Message Template.

    Returns:
        The application and message template configurations
    """
    return {
        "application_config": {
            "name": f"{_generate_application_name()}",
            "configuration": {
                "sendPinPerPhoneNumberLimit": "30/1d",  # Default value in Infobip: "3/1d"
            },
        },
        "message_config": {
            "register_phone_number": {
                "senderId": f"{os.environ['INFOBIP_SENDER_ID']}",
                "pinType": "NUMERIC",
                "messageText": "{{pin}} is your OTP to register the phone number. This OTP is valid for the next 15 mins.",
                "pinLength": 6,
            },
            "reset_password": {
                "senderId": f"{os.environ['INFOBIP_SENDER_ID']}",
                "pinType": "NUMERIC",
                "messageText": "{{pin}} is your OTP to reset the password. This OTP is valid for the next 15 mins.",
                "pinLength": 6,
            },
            "disable_mfa": {
                "senderId": f"{os.environ['INFOBIP_SENDER_ID']}",
                "pinType": "NUMERIC",
                "messageText": "{{pin}} is your OTP to disable the multi-factor authentication (MFA). This OTP is valid for the next 15 mins.",
                "pinLength": 6,
            },
            "get_token": {
                "senderId": f"{os.environ['INFOBIP_SENDER_ID']}",
                "pinType": "NUMERIC",
                "messageText": "{{pin}} is your OTP to get an application token. This OTP is valid for the next 15 mins.",
                "pinLength": 6,
            },
        },
    }

In [ ]:
config = get_application_and_message_config()
assert config["application_config"]["name"] == (
    os.environ["HOSTNAME"] or os.environ["DOMAIN"]
)

allowed_message_template_names = list(config["message_config"].keys())

assert [
    "register_phone_number",
    "reset_password",
    "disable_mfa",
    "get_token",
] == allowed_message_template_names, allowed_message_template_names
allowed_message_template_names

['register_phone_number', 'reset_password', 'disable_mfa', 'get_token']

In [ ]:
# | exporti


def _send_get_request_to_infobip(
    relative_url: str,
) -> Union[Dict[str, Any], List[Dict[str, Any]]]:
    """Send a GET request to Infobip

    Args:
        relative_url: The relative url of the infobip endpoint

    Returns:
        A list of JSON objects
    """

    headers = {
        "Authorization": f"App {os.environ['INFOBIP_API_KEY']}",
        "Accept": "application/json",
    }
    url = f"{os.environ['INFOBIP_BASE_URL']}{relative_url}"
    for i in range(3):
        err = None
        try:
            response = requests.get(url, headers=headers)
            if response:
                response_json = response.json()
                break
            else:
                err = response.text
                sleep(np.random.uniform(1, 3))

        except Exception as e:
            raise HTTPException(
                status_code=status.HTTP_503_SERVICE_UNAVAILABLE,
                detail=ERRORS["MESSAGING_SERVICE_UNAVAILABLE"],
            )
    if err:
        logger.exception(f"Unexpected response from infobip", exc_info=err)
        raise HTTPException(status_code=500, detail=f"Unexpected exception: {err}")

    return response_json  # type: ignore

In [ ]:
relative_url = "/2fa/2/applications"
applications = _send_get_request_to_infobip(relative_url)
assert isinstance(applications, list)

In [ ]:
# | exporti


def _get_application_id(application_name: str) -> Optional[str]:
    """Get application template id for the given application name

    Args:
        application_name: Application name for which the application id needs to be retrived

    Returns:
        The application id if the exists, else None
    """
    relative_url = "/2fa/2/applications"
    apps = _send_get_request_to_infobip(relative_url)
    app: List[Dict[str, Any]] = [app for app in apps if app["name"] == application_name]  # type: ignore

    return None if len(app) == 0 else app[0]["applicationId"]

In [ ]:
application_name = "random_name_for_testing"

expected = None
actual = _get_application_id(application_name)

display(actual)
assert actual == expected

None

In [ ]:
# | exporti


def _send_post_request_to_infobip(
    relative_url: str, data: Dict[str, Any]
) -> Dict[str, Any]:
    """Send a POST request to Infobip

    Args:
        relative_url: The relative url of the infobip endpoint
        data: Template configuration

    Returns:
        A JSON object of the response
    """

    headers = {
        "Authorization": f"App {os.environ['INFOBIP_API_KEY']}",
        "Content-Type": "application/json",
        "Accept": "application/json",
    }
    url = f"{os.environ['INFOBIP_BASE_URL']}{relative_url}"

    try:
        response: Dict[str, Any] = requests.post(url, json=data, headers=headers).json()
    except Exception as e:
        raise HTTPException(
            status_code=status.HTTP_503_SERVICE_UNAVAILABLE,
            detail=ERRORS["MESSAGING_SERVICE_UNAVAILABLE"],
        )
    return response

In [ ]:
# | exporti


def _create_all_message_templates(
    application_id: str, message_config: Dict[str, Dict[str, str]]
) -> Dict[str, str]:
    """Create new message templates with the given configurations

    Args:
        application_id: Application id for which the message templates needs to be created
        message_config: Message template configurations like message text, PIN type, PIN length etc.,

    Returns:
        The created message template name and their ids as a key value pair.
    """
    relative_url = f"/2fa/2/applications/{application_id}/messages"
    message_templates_to_ids = {}

    for template_name, config in message_config.items():
        message_templates_to_ids[template_name] = _send_post_request_to_infobip(
            relative_url, config
        )["messageId"]

    return message_templates_to_ids

In [ ]:
# Creating all message templates for the sample test application named "Test 2FA"

if os.environ["DOMAIN"] == "localhost":
    relative_url = "/2fa/2/applications"
    apps = _send_get_request_to_infobip(relative_url)

    if len(apps) > 0 and apps[0]["name"] == "Test 2FA":

        application_id = apps[0]["applicationId"]
        message_templates_to_ids = _create_all_message_templates(
            application_id, config["message_config"]
        )

        relative_url = f"/2fa/2/applications/{application_id}/messages"
        messages = _send_get_request_to_infobip(relative_url)

        msg_config = get_application_and_message_config()["message_config"]
        for k, v in message_templates_to_ids.items():
            message = [i for i in messages if i["messageId"] == v][0]
            display(message)
            assert (
                message["messageText"] == msg_config[k]["messageText"]
            ), f'{message["messageText"]=}, {msg_config[k]["messageText"]=}'

{'messageId': '23F5A8833224B9894DB8F52F87E5DD13',
 'applicationId': 'CC1BF4F8DC02A1EABF277DFC3E64671E',
 'pinPlaceholder': '{{pin}}',
 'messageText': '{{pin}} is your OTP to register the phone number. This OTP is valid for the next 15 mins.',
 'pinLength': 6,
 'pinType': 'NUMERIC',
 'senderId': 'InfoSMS',
 'speechRate': 1.0}

{'messageId': 'B76F69FDD729D54A78B6787B0A27AB07',
 'applicationId': 'CC1BF4F8DC02A1EABF277DFC3E64671E',
 'pinPlaceholder': '{{pin}}',
 'messageText': '{{pin}} is your OTP to reset the password. This OTP is valid for the next 15 mins.',
 'pinLength': 6,
 'pinType': 'NUMERIC',
 'senderId': 'InfoSMS',
 'speechRate': 1.0}

{'messageId': '70735F04BBA2E476DE5184C8A6802830',
 'applicationId': 'CC1BF4F8DC02A1EABF277DFC3E64671E',
 'pinPlaceholder': '{{pin}}',
 'messageText': '{{pin}} is your OTP to disable the multi-factor authentication (MFA). This OTP is valid for the next 15 mins.',
 'pinLength': 6,
 'pinType': 'NUMERIC',
 'senderId': 'InfoSMS',
 'speechRate': 1.0}

{'messageId': 'A1146F52686100205447A98E94937A90',
 'applicationId': 'CC1BF4F8DC02A1EABF277DFC3E64671E',
 'pinPlaceholder': '{{pin}}',
 'messageText': '{{pin}} is your OTP to get an application token. This OTP is valid for the next 15 mins.',
 'pinLength': 6,
 'pinType': 'NUMERIC',
 'senderId': 'InfoSMS',
 'speechRate': 1.0}

In [ ]:
# | exporti


def _get_message_id_for_template(
    xs: List[Dict[str, Union[str, int]]], message_template_name: str
) -> Optional[str]:
    """Get the message id for the given template

    Args:
        xs: The list of available message templates in Infobip server
        message_template_name: Message template name for which the id needs to be retrieved

    Returns:
        The message id if the template is already created, else None
    """
    config = get_application_and_message_config()

    message_ids = [
        x["messageId"]
        for x in xs
        if config["message_config"][message_template_name].items() <= x.items()
    ]

    if len(message_ids) == 0:
        return None

    return message_ids[0]  # type: ignore

In [ ]:
xs = [
    {
        "messageId": "000",
        "applicationId": "123",
        "pinPlaceholder": "{{pin}}",
        "messageText": "{{pin}} is your one-time verification code. The code is valid for the next 15 mins.",
        "pinLength": 6,
        "pinType": "NUMERIC",
        "senderId": "InfoSMS",
        "speechRate": 1.0,
    }
]
message_template_name = "register_phone_number"
actual = _get_message_id_for_template(xs, message_template_name)
expected = None
display(actual)
assert actual == expected

xs = [
    {
        "messageId": "123",
        "applicationId": "123",
        "pinPlaceholder": "{{pin}}",
        "messageText": "{{pin}} is your OTP to register the phone number. This OTP is valid for the next 15 mins.",
        "pinLength": 6,
        "pinType": "NUMERIC",
        "senderId": "InfoSMS",
        "speechRate": 1.0,
    }
]
message_template_name = "register_phone_number"
actual = _get_message_id_for_template(xs, message_template_name)
expected = "123"
display(actual)
assert actual == expected

None

'123'

In [ ]:
# | exporti


def _send_put_request_to_infobip(
    relative_url: str, data: Dict[str, Any]
) -> Dict[str, Any]:
    """Send a PUT request to Infobip

    Args:
        relative_url: The relative url of the infobip endpoint
        data: The updated template configuration

    Returns:
        A JSON object of the response
    """

    headers = {
        "Authorization": f"App {os.environ['INFOBIP_API_KEY']}",
        "Content-Type": "application/json",
        "Accept": "application/json",
    }
    url = f"{os.environ['INFOBIP_BASE_URL']}{relative_url}"

    try:
        response: Dict[str, Any] = requests.put(url, json=data, headers=headers).json()
    except Exception as e:
        raise HTTPException(
            status_code=status.HTTP_503_SERVICE_UNAVAILABLE,
            detail=ERRORS["MESSAGING_SERVICE_UNAVAILABLE"],
        )
    return response

In [ ]:
# | exporti


def _update_application_config(app_id: str, config: Dict[str, Any]) -> Dict[str, Any]:
    """Update an existing application configuration

    Args:
        app_id: Application ID for which the configurations needs to be updated
        config: New configuration for the application
    """
    relative_url = f"/2fa/2/applications/{app_id}"
    return _send_put_request_to_infobip(relative_url, config)

In [ ]:
# | exporti


def _is_config_changed(
    local_config: Dict[str, Any], server_config: Dict[str, Any]
) -> bool:
    """Verify that the shared key values are different for the given configurations

    Args:
        local_config: The template configuration set in the code
        server_config: The template configuration retrieved from the infobip server

    Returns:
        True, if the configurations didn't match else False
    """

    shared_keys = local_config.keys() & server_config.keys()
    for k in shared_keys:
        if not local_config[k] == server_config[k]:
            return True

    return False

In [ ]:
local_config = {
    "sendPinPerPhoneNumberLimit": "30/1d",
    "pinAttempts": 10,
    "verifyPinLimit": "1/3s",
}
server_config = {
    "pinAttempts": 10,
    "allowMultiplePinVerifications": False,
    "pinTimeToLive": "15m",
    "verifyPinLimit": "1/3s",
    "sendPinPerApplicationLimit": "10000/1d",
    "sendPinPerPhoneNumberLimit": "10/1d",
}

actual = _is_config_changed(local_config, server_config)
display(actual)
assert actual

True

In [ ]:
local_config = {"sendPinPerPhoneNumberLimit": "30/1d", "pinAttempts": 10}
server_config = {
    "pinAttempts": 10,
    "allowMultiplePinVerifications": False,
    "pinTimeToLive": "15m",
    "verifyPinLimit": "1/3s",
    "sendPinPerApplicationLimit": "10000/1d",
    "sendPinPerPhoneNumberLimit": "30/1d",
}

actual = _is_config_changed(local_config, server_config)
display(actual)
assert not actual

False

In [ ]:
local_config = {
    "messageText": "{{pin}} is your OTP to register the phone number. This OTP is valid for the next 15 mins.",
    "pinLength": 6,
}
server_config = {
    "messageId": "60583E51E79AA5AF2A0D9E974D648689",
    "applicationId": "12D2CA96F21A5ED26508CAAEB221A4C4",
    "pinPlaceholder": "{{pin}}",
    "messageText": "{{pin}} is your OTP to register the phone number. This OTP is valid for the next 15 mins.",
    "pinLength": 6,
    "pinType": "NUMERIC",
    "senderId": "InfoSMS",
    "speechRate": 1.0,
}
actual = _is_config_changed(local_config, server_config)
display(actual)
assert not actual

False

In [ ]:
# | export


def get_app_and_message_id(message_template_name: str) -> Tuple[str, str]:
    """Get the application id and the message template id

    A new application and a message template will be created in the Infobip server if the requested
    templates are not created yet. In case, the templates are already created then they will be reused.

    Args:
        message_template_name: Message template name for which the id needs to be retrieved. Currently,
            the API only supports **register_phone_number** and **reset_password** as message template name.

    Returns:
        The application id and the message template id for the given message template
    """
    config = get_application_and_message_config()
    application_id = _get_application_id(config["application_config"]["name"])

    if application_id is not None:
        # check and update application configuration if the local configurations are not matching the server configurations
        relative_url = f"/2fa/2/applications/{application_id}"
        server_app_config = _send_get_request_to_infobip(relative_url)["configuration"]  # type: ignore
        local_app_config = config["application_config"]["configuration"]

        if _is_config_changed(local_app_config, server_app_config):
            application_id = _update_application_config(
                application_id, config["application_config"]
            )["applicationId"]

        # Get id for the given message_template_name
        relative_url = f"/2fa/2/applications/{application_id}/messages"
        messages = _send_get_request_to_infobip(relative_url)
        message_id = _get_message_id_for_template(messages, message_template_name)  # type: ignore

        if message_id is None:
            # Create a new message template for the message_template_name
            relative_url = f"/2fa/2/applications/{application_id}/messages"
            ret_val_message_id = _send_post_request_to_infobip(
                relative_url, config["message_config"][message_template_name]
            )["messageId"]
        else:
            ret_val_message_id = message_id
    else:
        # Create a new application template
        relative_url = "/2fa/2/applications"
        application_id = _send_post_request_to_infobip(
            relative_url, config["application_config"]
        )["applicationId"]

        # Create new message templates
        message_templates_to_ids = _create_all_message_templates(
            application_id, config["message_config"]
        )
        ret_val_message_id = message_templates_to_ids[message_template_name]

    return application_id, ret_val_message_id  # type: ignore

In [ ]:
def create_and_test_new_templates(message_template_name: str):
    application_id_1, message_id_1 = get_app_and_message_id(message_template_name)
    application_id_2, message_id_2 = get_app_and_message_id(message_template_name)

    assert application_id_1 == application_id_2
    assert message_id_1 == message_id_2

    display(application_id_1, message_id_1)

    # List all app templates make make sure we have only one entry of the newly created app template
    relative_url = "/2fa/2/applications"
    apps = _send_get_request_to_infobip(relative_url)
    apps = [i["applicationId"] for i in apps]

    assert apps.count(application_id_1) == 1

    # make sure the application configs are matching
    relative_url = f"/2fa/2/applications/{application_id_1}"
    server_app_config = _send_get_request_to_infobip(relative_url)["configuration"]
    config = get_application_and_message_config()
    local_app_config = config["application_config"]["configuration"]

    assert not _is_config_changed(local_app_config, server_app_config)

    # List all messages make make sure we have only one entry of the newly created message templates
    relative_url = f"/2fa/2/applications/{application_id_1}/messages"
    messages = _send_get_request_to_infobip(relative_url)

    message_ids = [i["messageId"] for i in messages]
    assert message_ids.count(message_id_1) == 1

    relative_url = f"/2fa/2/applications/{application_id_1}/messages/{message_id_1}"
    server_msg_config = _send_get_request_to_infobip(relative_url)
    local_msg_config = config["message_config"][message_template_name]

    assert not _is_config_changed(local_msg_config, server_msg_config)


if os.environ["DOMAIN"] == "localhost":
    for message_template_name in config["message_config"].keys():
        display(f"\nApplication id and message id for: {message_template_name}")
        create_and_test_new_templates(message_template_name=message_template_name)

'\nApplication id and message id for: register_phone_number'

'12D2CA96F21A5ED26508CAAEB221A4C4'

'60583E51E79AA5AF2A0D9E974D648689'

'\nApplication id and message id for: reset_password'

'12D2CA96F21A5ED26508CAAEB221A4C4'

...

'12D2CA96F21A5ED26508CAAEB221A4C4'

'C168073E779C6B2E5A2C097C6FDFA52B'

'\nApplication id and message id for: get_token'

'12D2CA96F21A5ED26508CAAEB221A4C4'

'317DEE435D439EFCEF3314E916A26081'

In [ ]:
# | export


def send_sms(application_id: str, message_id: str, phone_number: str) -> Dict[str, Any]:
    """Send a OTP code over SMS using infobip services

    Args:
        application_id: The ID of the application that represents the 2FA service,
        message_id: The ID of the message template to send to the recipient.
        phone_number: The phone number of the recipient.

    Returns:
        A JSON object of the response from Infobip
    """
    relative_url = "/2fa/2/pin"
    data = {
        "applicationId": f"{application_id}",
        "messageId": f"{message_id}",
        "to": f"{phone_number}",
    }
    return _send_post_request_to_infobip(relative_url, data)

In [ ]:
# | export


def verify_pin(pin_id: str, otp: str) -> Dict[str, Any]:
    """Verify a phone number using infobip services.

    Args:
        pin_id: The ID of the pin that needs to be verified. The Infobip 2FA service will dynamically create a pin while sending the SMS.
            And while verifying the OTP, we need to send the same pin to the Infobip 2FA service to validate on their servers.
        otp: The OTP sent to the user via SMS.

    Returns:
        A JSON object of the response from Infobip
    """
    relative_url = f"/2fa/2/pin/{pin_id}/verify"
    data = {
        "pin": otp,
    }
    return _send_post_request_to_infobip(relative_url, data)

In [ ]:
# | exporti


def validate_otp(
    user: User,
    otp: str,
    message_template_name: str,
    session: Session,
) -> None:
    """Validate the SMS OTP

    Args:
        user: User object for whom the SMS OTP needs to be checked
        otp: The SMS OTP to validate
        message_template_name: Message template name to validate the OTP
        session: Session object

    Raises:
        HTTPException: If the Phone number is not registered and not verified
        HTTPException: If the OTP is invalid
    """
    application_id, message_id = get_app_and_message_id(
        message_template_name=message_template_name
    )

    sms = session.exec(
        select(SMS)
        .where(SMS.user == user)
        .where(SMS.application_id == application_id)
        .where(SMS.message_id == message_id)
    ).one_or_none()
    if sms is None:
        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail=ERRORS["PHONE_NUMBER_NOT_REGISTERED"],
        )

    sms_protocol = session.exec(
        select(SMSProtocol).where(SMSProtocol.sms_id == sms.id)
    ).one_or_none()

    if sms_protocol is None:
        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail=ERRORS["PHONE_NUMBER_NOT_REGISTERED"],
        )

    if sms_protocol.pin_attempts_remaining == 0:
        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail=ERRORS["NO_MORE_PIN_ATTEMPTS"],
        )

    pin_verification_status = airt_service.sms_utils.verify_pin(
        sms_protocol.pin_id, otp
    )

    if "requestError" in pin_verification_status:
        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail=f"{pin_verification_status['requestError']['serviceException']['text']}",
        )

    if not pin_verification_status["verified"]:
        with commit_or_rollback(session):
            sms_protocol.pin_verified = pin_verification_status["verified"]
            sms_protocol.pin_attempts_remaining = pin_verification_status[
                "attemptsRemaining"
            ]
            session.add(sms_protocol)

        raise HTTPException(
            status_code=status.HTTP_400_BAD_REQUEST,
            detail=ERRORS[f"{pin_verification_status['pinError']}"],
        )

    with commit_or_rollback(session):
        session.delete(sms_protocol)

In [ ]:
with get_session_with_context() as session:
    test_user = create_user_for_testing()
    user = session.exec(select(User).where(User.username == test_user)).one()
    message_template_name = "get_token"
    random_otp = "000000"
    with pytest.raises(HTTPException) as e:
        validate_otp(
            user=user,
            otp=random_otp,
            message_template_name=message_template_name,
            session=session,
        )
    display(e.value.detail)

'The phone number is not yet registered. Please register your phone number before calling this method.'

In [ ]:
# Monkey patching positive scenario
test_username = create_user_for_testing()
random_phone_number = "910000000000"
random_sms_pin_id = "my_random_pin_id"
message_template_name = "disable_mfa"
random_sms_otp = "111111"
with MonkeyPatch.context() as monkeypatch:

    send_sms_sample_response = {
        "pinId": random_sms_pin_id,
        "to": random_phone_number,
        "ncStatus": "NC_NOT_CONFIGURED",
        "smsStatus": "MESSAGE_SENT",
    }
    monkeypatch.setattr(
        "airt_service.sms_utils.send_sms", lambda x, y, z: send_sms_sample_response
    )

    verify_pin_sample_response = {
        "pinId": random_sms_pin_id,
        "msisdn": random_phone_number,
        "verified": True,
        "attemptsRemaining": 0,
    }

    monkeypatch.setattr(
        "airt_service.sms_utils.verify_pin", lambda x, y: verify_pin_sample_response
    )

    with get_session_with_context() as session:
        user = session.exec(select(User).where(User.username == test_username)).one()
        user.phone_number = random_phone_number
        user.is_phone_number_verified = True
        session.add(user)
        session.commit()
        session.refresh(user)

        actual = generate_mfa_url(user=user, session=session)
        assert user.mfa_secret is not None
        # activate MFA
        activate_mfa_request = ActivateMFARequest(
            user_otp=pyotp.TOTP(user.mfa_secret).now()
        )
        actual = activate_mfa(
            activate_mfa_request=activate_mfa_request, user=user, session=session
        )
        assert actual.is_mfa_active
        display(actual)

        actual = send_sms_otp(
            username=test_username,
            message_template_name=message_template_name,
            session=session,
        )
        display(actual)

        random_otp = "000000"
        validate_otp(
            user=user,
            otp=random_otp,
            message_template_name=message_template_name,
            session=session,
        )
        display("Ok")

User(id=10, uuid=UUID('1b69e42e-6f02-4dd3-ad94-a7cedc657413'), username='vbcmsxtapn', first_name='unittest', last_name='user', email='vbcmsxtapn@email.com', subscription_type=<SubscriptionType.test: 'test'>, super_user=False, disabled=False, created=datetime.datetime(2022, 10, 21, 5, 14, 16), phone_number='910000000000', is_phone_number_verified=True, mfa_secret=**********************************, is_mfa_active=True)

'If you have already registered and verified your phone number, you will receive the OTP by SMS. If you did not receive the OTP, please contact your administrator.'

'Ok'